In [2]:
import seaborn as sns
import pandas as pd 
import datetime as datetime
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import numpy as np
# Read data from file 'filename.csv' |
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later)
pd.options.display.float_format = '{:,.2f}'.format
data = pd.read_csv("/home/anconam/projects/Shapley_Pruning/experiments/results/log.csv") 
data['timestamp'].apply(lambda row: str(row))


data.head()

,timestamp,epoch,train_acc,test_acc,train_loss,test_loss,n_params,flops,n_params_full,activations,train_time,prune_time,experiment
0,20200119T151413,1,0.84,0.88,0.46,0.36,"156,459,008.00","16,849,994.00","156,459,008.00",4096-64-32,11.10,28.75,fmnist__sv-abs-smallest_gap:6.0_steps:1_start:...
1,20200119T152102,1,0.84,0.88,0.46,0.36,"156,459,008.00","16,849,994.00","156,459,008.00",4096-64-32,11.09,6.95,fmnist__count-smallest_gap:6.0_steps:1_start:1...
2,20200119T152131,1,0.84,0.87,0.46,0.36,"156,459,008.00","16,849,994.00","156,459,008.00",4096-64-32,11.10,6.89,fmnist__grad-smallest_gap:6.0_steps:1_start:1_...
3,20200119T152159,1,0.84,0.88,0.46,0.34,"156,459,008.00","16,849,994.00","156,459,008.00",4096-64-32,11.26,6.93,fmnist__taylor-smallest_gap:6.0_steps:1_start:...
4,20200119T152224,1,0.84,0.89,0.46,0.34,"156,459,008.00","16,849,994.00","156,459,008.00",4096-64-32,11.09,6.84,fmnist__weight-smallest_gap:6.0_steps:1_start:...


### FMNIST CNN 1.8M parameters

In [3]:
dataset = "fmnist"

plot_df = data[data["experiment"].str.startswith(dataset)]
# plot_df = plot_df[plot_df["timestamp"] >"20200116T18"]

# plot_df = plot_df[plot_df["timestamp"] > "20200117T09"]
# plot_df = plot_df[~plot_df["experiment"].str.contains("prunedyn_sv")]
# plot_df = plot_df[~plot_df["experiment"].str.contains("in1ep")]
# plot_df = plot_df[~plot_df["experiment"].str.contains("random")]
# plot_df = plot_df[~plot_df["experiment"].str.contains("30steps")]

# Some processing
plot_df['test_acc'] *= 100
plot_df['train_acc'] *= 100
plot_df["total_time"] = plot_df["train_time"] + plot_df["prune_time"]
plot_df["sparsity"] = plot_df["n_params"] / plot_df["n_params_full"] * 100
plot_df["test_acc_gap"] = (plot_df["test_acc"] - plot_df["test_acc"].max())


In [4]:
# Rank pruning methods by Test Accuracy at the epoch of Minimum Test Loss
# df_join = plot_df[plot_df["sparsity"] < 100]
# df_join = df_join[df_join["epoch"] > 25]

df_join = plot_df.join(plot_df.groupby(["timestamp"])["test_loss"].min(), on="timestamp", rsuffix="_min")
df_join = df_join[df_join["test_loss"] == df_join["test_loss_min"]]
std = df_join.groupby(["experiment"]).std().sort_values(by=["test_acc"], ascending=False)
df_join = df_join.groupby(["experiment"]).mean().sort_values(by=["test_acc"], ascending=False)
df_join["test_acc_std"] = std["test_acc"]
df_join.filter(["experiment", "test_acc","test_acc_std", "test_acc_gap", "sparsity", "epoch", "test_loss"])

,test_acc,test_acc_std,test_acc_gap,sparsity,epoch,test_loss
experiment,,,,,,
fmnist__weight-smallest_gap:6.0_steps:1_start:1_int:1,88.52,nan,0.00,100.00,1,0.34
fmnist__sv-abs-smallest_gap:6.0_steps:1_start:1_int:1,88.37,nan,-0.15,100.00,1,0.36
fmnist__taylor-smallest_gap:6.0_steps:1_start:1_int:1,88.06,nan,-0.46,100.00,1,0.34
fmnist__count-smallest_gap:6.0_steps:1_start:1_int:1,87.92,nan,-0.60,100.00,1,0.36
fmnist__random_gap:6.0_steps:1_start:1_int:1,87.68,nan,-0.84,100.00,1,0.36
fmnist__grad-smallest_gap:6.0_steps:1_start:1_int:1,86.88,nan,-1.64,100.00,1,0.36


In [5]:
# Rank pruning methods by best test accuracy during training and sparsity at that stage
# df_join = plot_df[plot_df["sparsity"] < 100]
# df_join = df_join[df_join["epoch"] > 25]

df_join = plot_df[plot_df["sparsity"] < 101]
df_join = df_join.join(df_join.groupby(["timestamp"])["test_acc"].max(), on="timestamp", rsuffix="_max")
df_join = df_join[df_join["test_acc"] == df_join["test_acc_max"]]
std = df_join.groupby(["experiment"]).std().sort_values(by=["test_acc"], ascending=False)
df_join = df_join.groupby(["experiment"]).mean().sort_values(by=["test_acc"], ascending=False)
df_join["test_acc_std"] = std["test_acc"]

df_join.filter(["experiment", "test_acc","test_acc_std", "test_acc_gap", "sparsity", "epoch", "test_loss"])

,test_acc,test_acc_std,test_acc_gap,sparsity,epoch,test_loss
experiment,,,,,,
fmnist__weight-smallest_gap:6.0_steps:1_start:1_int:1,88.52,nan,0.00,100.00,1,0.34
fmnist__sv-abs-smallest_gap:6.0_steps:1_start:1_int:1,88.37,nan,-0.15,100.00,1,0.36
fmnist__taylor-smallest_gap:6.0_steps:1_start:1_int:1,88.06,nan,-0.46,100.00,1,0.34
fmnist__count-smallest_gap:6.0_steps:1_start:1_int:1,87.92,nan,-0.60,100.00,1,0.36
fmnist__random_gap:6.0_steps:1_start:1_int:1,87.68,nan,-0.84,100.00,1,0.36
fmnist__grad-smallest_gap:6.0_steps:1_start:1_int:1,86.88,nan,-1.64,100.00,1,0.36


In [6]:
plot_df = plot_df[plot_df["experiment"].str.contains("gap:10")]

print ("Accuracy")
performance_df = plot_df.groupby(['experiment', 'timestamp']).max()
performance_df = performance_df.groupby(['experiment']).mean()
# print (performance_df.filter(items=["experiment", "test_acc"]))

# Print train time
print ("Training time")
time_df = plot_df.groupby(['experiment', 'timestamp']).sum()
time_df = time_df.groupby(['experiment']).mean()
cum_time_df = plot_df
cum_time_df['total_time_cum'] = plot_df.groupby(['experiment', 'timestamp'])['total_time'].transform(pd.Series.cumsum)
# print (cum_time_df)
# print (time_df.filter(items=["experiment", "train_time", "prune_time", "total_time"]))

# Weight sparsity
print ("Sparsity")
initial_params = np.max(list(plot_df.groupby(['experiment'])["n_params"].max()))
# print (initial_params)
sparsity_df = plot_df.groupby(['experiment', 'timestamp']).min()
sparsity_df = sparsity_df.groupby(['experiment']).min()
# print (sparsity_df.filter(items=["experiment", "n_params"]) / initial_params * 100)


# for label in plotting_data:
plt.figure(figsize=(12,8))
sns.lineplot(x="epoch", y="test_acc", hue="experiment", data = plot_df)
plt.title("Accuracy")
plt.ylim([0.0, 1.0])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.figure(figsize=(12,8))
new_plot = plot_df
new_plot["train_time_cum"] = new_plot.groupby(['experiment', 'timestamp'])['train_time'].apply(lambda x: x.cumsum())
sns.lineplot(x="train_time_cum", y="test_acc", hue="experiment", data = new_plot)
plt.title("Accuracy / Wall-clock Time")
plt.ylim([0.8, 0.98])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.figure(figsize=(12,8))
ax = sns.lineplot(x="epoch", y="test_loss", hue="experiment", data = plot_df)
plt.title("Loss")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.figure(figsize=(12,8))
ax = sns.lineplot(x="epoch", y="train_loss", hue="experiment", data = plot_df)
plt.title("Train Loss")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.figure(figsize=(12,8))
ax = sns.lineplot(x="epoch", y="total_time_cum", hue="experiment", data = cum_time_df)
plt.title("Train Time / Epoch")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.figure(figsize=(12,8))
ax = sns.lineplot(x="epoch", y="n_params", hue="experiment", data = plot_df)
plt.title("# Parameters")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


Accuracy


KeyError: 'experiment'